In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [6]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # detection
        result = pose.process(image)
        
        # color back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # detection
        result = pose.process(image)
        
        # color back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try: 
            landmarks = result.pose_landmarks.landmark
        except:
            pass
        
        
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [11]:
len(landmarks)

33

In [2]:
def calc_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle

In [ ]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # detection
        result = pose.process(image)
        
        # color back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try: 
            landmarks = result.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calc_angle(shoulder, elbow, wrist)

            coord = tuple(np.multiply(elbow, [640, 480]))
            print(coord)
            print(str(int(angle)))            
            
            cv2.putText(image, str(int(angle)), (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

        except:
            pass
        
        
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
vcap = cv2.VideoCapture('KneeBendVideo.mp4')
pose = mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5)

  
width  = vcap.get(3)    
height = vcap.get(4)

start = 0
end = 0
rep_count = 0

while(True):
    ret, frame = vcap.read()
    cv2.imshow('frame',frame)
   
    if frame is not None:    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        result = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        try:
            landmarks = result.pose_landmarks.landmark

            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            angle = calc_angle(ankle, knee, hip)

            coord = tuple(np.multiply(knee, [width, height]).astype(int))

            cv2.putText(image, str(int(angle)), coord, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

            if angle < 140:
                if start == 0:
                    start = time.time()
            if angle >= 140:
                end = time.time()
                if(start != 0):
                    rep_time = end - start
                    if rep_time < 8:
                        cv2.putText(image, "Keep your knee bent", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
                    else:
                        rep_count += 1
                    start = end = 0
        except:
            start = end = 0
            pass
        
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.putText(image, "Rep Count: " + str(rep_count), (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('Video', image)
    else:
        print("Frame is none")
        
    if cv2.waitKey(22) & 0xFF == ord('q'):
        break

vcap.release()
cv2.destroyAllWindows()
print("Video stop")


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Video stop


In [5]:
landmarks[25]

x: 0.4981696307659149
y: 0.8737692832946777
z: -0.1625491827726364
visibility: 0.8063628673553467

In [6]:
landmarks[26]

x: 0.48090246319770813
y: 0.7841046452522278
z: 0.16334481537342072
visibility: 0.28044819831848145

In [7]:
landmarks[23]

x: 0.6518848538398743
y: 0.8206548690795898
z: -0.11039306968450546
visibility: 0.9990280866622925

In [8]:
landmarks[24]

x: 0.6237873435020447
y: 0.8084115982055664
z: 0.11068693548440933
visibility: 0.998873770236969

In [9]:
landmarks[27]

x: 0.40726232528686523
y: 0.8597673177719116
z: 0.04737330973148346
visibility: 0.3518066108226776

In [10]:
landmarks[28]

x: 0.40160632133483887
y: 0.8548696637153625
z: 0.19516511261463165
visibility: 0.19113965332508087

In [11]:
left_hip = landmarks[23]

In [12]:
left_hip.y

0.8206548690795898